In [19]:
import os

# Get the current working directory
cwd = os.getcwd()
print(f"Current working directory: {cwd}")

# Check if the current working directory ends with "notebooks"
if cwd.endswith("notebooks"):
    # Set the cwd to the parent directory
    parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))
    os.chdir(parent_dir)
    print(f"Changed working directory to parent: {parent_dir}")
else:
    print("No change in working directory needed")


Current working directory: /Users/apple/Documents/dagster-bio
No change in working directory needed


In [46]:
from enum import unique
from pydantic import BaseModel
import subprocess
from typing import List


class Dependency(BaseModel):
    channel: str
    package: str
    version: str


class CondaImage(BaseModel):
    image_name: str
    image_version: str
    dependencies: List[Dependency]

    def get_docker_build_cmd(self):
        # Construct the Docker build command with formatted build arguments
        docker_build_cmd = [
            "docker",
            "build",
            "-t",
            f"{self.image_name}:{self.image_version}",
            "-f",
            "dagster_bio/Dockerfile.conda",
        ]

        # Format CONDA_DEPENDENCIES build argument
        if self.dependencies:
            dependencies_str = " ".join(
                [f"{dep.channel}::{dep.package}={dep.version}" for dep in self.dependencies]
            )
            dependencies_str = f'"{dependencies_str}"'
            docker_build_cmd.extend(
                ["--build-arg", f'CONDA_DEPENDENCIES={dependencies_str}']
            )

        docker_build_cmd.append(".")

        return " ".join(docker_build_cmd)

    def build(self):
        # Get Docker build command
        docker_build_cmd = self.get_docker_build_cmd()

        # Open a subprocess to execute the Docker build command
        proc = subprocess.Popen(
            docker_build_cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,  # Ensures output is treated as text (Python 3.7+)
        )

        # Read and print subprocess output line by line
        while True:
            stdout_line = proc.stdout.readline()
            stderr_line = proc.stderr.readline()
            if not stdout_line and not stderr_line and proc.poll() is not None:
                break
            if stdout_line:
                print(stdout_line.strip())
            if stderr_line:
                print(stderr_line.strip())

        # Ensure all output is read
        stdout, stderr = proc.communicate()
        if stdout:
            print(stdout.strip())
        if stderr:
            print(stderr.strip())

        # Check the exit code
        return proc.returncode

# Example usage:

In [50]:
prodigal_image = CondaImage(
    image_name="basescan-prodigal",
    image_version="1.0",
    dependencies=[
        Dependency(channel="bioconda", package="prodigal", version="2.6.3-8"),
        # Dependency(channel="conda-forge", package="pandas", version="1.3.3"),
    ],
)

print(prodigal_image.get_docker_build_cmd())

docker build -t basescan-prodigal:1.0 -f dagster_bio/Dockerfile.conda --build-arg CONDA_DEPENDENCIES="bioconda::prodigal=2.6.3-8" .


In [51]:
prodigal_image.build()

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile.conda
#1 transferring dockerfile: 397B 0.0s done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/mambaorg/micromamba:0.25.1
#2 DONE 0.7s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [1/3] FROM docker.io/mambaorg/micromamba:0.25.1@sha256:60f50fe56fdb7f5c41c494ea79b4fce7f4accb02d72f3eb0bca9d1369e5b40a9
#4 CACHED

#5 [2/3] RUN micromamba create -n conda_env
#5 0.392
#5 0.392                                            __
#5 0.392           __  ______ ___  ____ _____ ___  / /_  ____ _
#5 0.392          / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
#5 0.392         / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
#5 0.392        / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
#5 0.392       /_/
#5 0.392
#5 0.394 Empty environment created at prefix: /opt/conda/envs/conda_env
#5 DONE 0.4s

#6 [3/3] RUN eval "$(micromamba shell hook --shell

0

In [49]:
prodigal_image.get_docker_build_cmd()

'docker build -t basescan-prodigal:1.0 -f dagster_bio/Dockerfile.conda --build-arg CONDA_DEPENDENCIES="bioconda::prodigal=2.6.3-8 conda-forge::pandas=1.3.3" .'